In [6]:
import tensorflow as tf
import numpy as np
import cv2
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator

layers = tf.keras.layers

In [7]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true" 

In [8]:
#wget  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
#!tar -xvf imagenette2.tgz

In [9]:
train_len = 9469
valid_len = 3925/16

In [10]:
# create a new generator
imagegen = ImageDataGenerator()
# load train data
train = imagegen.flow_from_directory("imagenette2/train/", class_mode="sparse", shuffle=True, batch_size=16, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory("imagenette2/val/", class_mode="sparse", shuffle=True, batch_size=16, target_size=(224, 224))

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [11]:
class ResNet(tf.keras.Model):
  def __init__(self):
    super(ResNet,self).__init__()

    self.conv1 = layers.Conv2D(64,7,2,activation='relu',kernel_initializer='he_normal')
    self.z_padding_1 = layers.ZeroPadding2D(padding=(3, 3))
    self.z_padding_2 = layers.ZeroPadding2D(padding=(1, 1))
    self.max_pool_1 = layers.MaxPool2D(3,strides=2)
    self.bn_1  = layers.BatchNormalization()
    self.relu_1 = layers.Activation('relu')
    self.global_pooling = layers.GlobalMaxPooling2D()
    #self.fc_1 = layers.Dense(2048,activation='softmax')
    #self.fc_2 = layers.Dense(2048,activation='softmax')
    self.fc_class = layers.Dense(10,activation='softmax')


  def conv_block(self,x,filers_sizes,kernel_sizes):

    def batchNorm_relu(x):

      x = layers.BatchNormalization()(x)
      x = layers.Activation('relu')(x)

      return x


    #Shortcut
    shortcut = x
    shortcut = layers.Conv2D(kernel_sizes[2],1,strides=2,kernel_initializer='he_normal')(shortcut)

    #Forward
    x = layers.Conv2D(filers_sizes[0],kernel_sizes[0],strides=2,kernel_initializer='he_normal')(x)
    x = batchNorm_relu(x)
    x = layers.Conv2D(filers_sizes[1],kernel_sizes[1],padding='same',kernel_initializer='he_normal')(x)
    x = batchNorm_relu(x)
    x = layers.Conv2D(filers_sizes[2],kernel_sizes[2],kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([x,shortcut]) 
    x = layers.Activation('relu')(x)

    return x

  def identity_block(self,x,filers_sizes,kernel_sizes):

    def batchNorm_relu(x):
      # BATH NORMALIZATION + RELU 
      x = layers.BatchNormalization()(x)
      x = layers.Activation('relu')(x)

      return x


    #Shortcut
    shortcut = x

    #Forward
    x = layers.Conv2D(filers_sizes[0],kernel_sizes[0],kernel_initializer='he_normal')(x)
    x = batchNorm_relu(x)
    x = layers.Conv2D(filers_sizes[1],kernel_sizes[1],padding='same',kernel_initializer='he_normal')(x)
    x = batchNorm_relu(x)
    x = layers.Conv2D(filers_sizes[2],kernel_sizes[2],kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([x,shortcut]) 
    x = layers.Activation('relu')(x)

    return x

  def call(self,x):

    x = self.z_padding_1(x)
    x = self.conv1(x)
    x = self.bn_1(x)
    x = self.relu_1(x)
    x = self.z_padding_2(x)   
    x = self.max_pool_1(x)
    
    x = self.conv_block(x,[64,64,256],[1,3,1])
    x = self.identity_block(x,[64,64,256],[1,3,1])
    x = self.identity_block(x,[64,64,256],[1,3,1])

    x = self.conv_block(x,[128,128,512],[1,3,1])
    x = self.identity_block(x,[128,128,512],[1,3,1])
    x = self.identity_block(x,[128,128,512],[1,3,1])
    x = self.identity_block(x,[128,128,512],[1,3,1])

    x = self.conv_block(x,[256,256,1024],[1,3,1])
    x = self.identity_block(x,[256,256,1024],[1,3,1])
    x = self.identity_block(x,[256,256,1024],[1,3,1])
    x = self.identity_block(x,[256,256,1024],[1,3,1])
    x = self.identity_block(x,[256,256,1024],[1,3,1])
    x = self.identity_block(x,[256,256,1024],[1,3,1])

    x = self.conv_block(x,[512,512,2048],[1,3,1])
    x = self.identity_block(x,[512,512,2048],[1,3,1])
    x = self.identity_block(x,[512,512,2048],[1,3,1])

    x = self.global_pooling(x)
    #x = self.fc_1(x)
    #x = self.fc_2(x)
    x = self.fc_class(x)

    return x

In [12]:
model = ResNet()

In [13]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

optimizer   = tf.keras.optimizers.SGD(lr=0.01,momentum=0.9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [14]:

def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
    
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [15]:

def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 100
ITER = 0
ITER_MAX = 246

for epoch in range(EPOCHS):

  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train:
    train_step(images, labels)
    images = images/255.0
    ITER += 16
    if ITER >= ITER_MAX:
        ITER = 0
        break 

  for test_images, test_labels in val:
    test_step(test_images, test_labels)
    ITER += 16
    if ITER >= ITER_MAX:
        ITER = 0
        break 

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 14.39212703704834, Accuracy: 10.546875, Test Loss: 14.795907974243164, Test Accuracy: 8.203125
Epoch 2, Loss: 14.418140411376953, Accuracy: 10.546875, Test Loss: 14.669984817504883, Test Accuracy: 8.984375
Epoch 3, Loss: 14.6699857711792, Accuracy: 8.984375, Test Loss: 14.292217254638672, Test Accuracy: 11.328125
Epoch 4, Loss: 14.040371894836426, Accuracy: 12.890625, Test Loss: 14.040372848510742, Test Accuracy: 12.890625
Epoch 5, Loss: 14.79590892791748, Accuracy: 8.203125, Test Loss: 15.110715866088867, Test Accuracy: 6.25
Epoch 6, Loss: 14.229256629943848, Accuracy: 11.71875, Test Loss: 14.292217254638672, Test Accuracy: 11.328125
